In [2]:
!pip install pandas google-cloud-bigquery
!pip install pandas-gbq


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:

import pandas as pd
from google.oauth2 import service_account
import pandas as pd
import pandas_gbq as gbq
import json
from pandas_gbq import to_gbq

credentials_path = '../token.json'

# Authenticate with your credentials
credentials = service_account.Credentials.from_service_account_file(
    credentials_path, scopes=['https://www.googleapis.com/auth/bigquery'])

# Set the credentials for pandas_gbq
gbq.context.credentials = credentials

In [31]:
with open('../token.json', 'r') as token_file:
    token_data = json.load(token_file)
    project_id = token_data.get('project_id', 'default-project-id')

dataset_id = "capstone"
table_id = 'cri_data'

to_gbq(cri_data, destination_table=f'{project_id}.{dataset_id}.{table_id}', project_id=project_id, if_exists='replace')

ArrowTypeError: Expected a string or bytes dtype, got int64

In [32]:
cri_data = pd.read_pickle('xy_data.pkl')

In [29]:
cri_data = cri_data.drop(['Duration_days'], axis =1)

In [3]:
cri_data = pd.read_pickle('xy_data_6months.pkl')

In [6]:
cri_data[cri_data['Default'] == 1].shape[0]

36234

In [7]:
cri_data.shape[0]

2240223

In [4]:
compustat_data = pd.read_pickle('compustat_inputed_v8.pkl')

In [10]:
compustat_data.shape

(784752, 30)

In [5]:
# convert to monthly 
def expand_dates(ser):
    min_date = ser['datadate'].min()
    max_date = ser['datadate'].max()
    return pd.DataFrame({'datadate': pd.date_range(min_date, max_date, freq='M')})

# Apply the function within each 'id' group
ffdf = compustat_data.groupby(['gvkey']).apply(expand_dates).reset_index()

# Merge with the original DataFrame and forward fill missing values for the entire row
result = ffdf.merge(compustat_data, on=['gvkey', 'datadate'], how='left').fillna(method='ffill')
result['mm'] = result['datadate'].dt.month
result.drop('level_1', axis=1, inplace=True)
result.head()

C:\Users\shiho\AppData\Local\Temp\ipykernel_40052\173701593.py:11: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  result = ffdf.merge(compustat_data, on=['gvkey', 'datadate'], how='left').fillna(method='ffill')


,gvkey,datadate,sic,tic,year,quarter,current_ratio,quick_ratio,cash_ratio,net_working_capital,...,working_capital_turnover,price_to_earnings,dividend_payout_ratio,retention_ratio,gross_margin_ratio,operating_profit_margin,ebitda_margin,debt_service_coverage_ratio,interest_coverage_ratio,mm
0,1004,2000-02-29,5080.0,AIR,2000.0,1.0,2.981531,1.176928,-0.021609,350.202,...,0.616739,59.375000,0.212500,0.787500,18.234795,-72.491564,-70.807951,-0.958372,4.223950,2
1,1004,2000-03-31,5080.0,AIR,2000.0,1.0,2.981531,1.176928,-0.021609,350.202,...,0.616739,59.375000,0.212500,0.787500,18.234795,-72.491564,-70.807951,-0.958372,4.223950,3
2,1004,2000-04-30,5080.0,AIR,2000.0,1.0,2.981531,1.176928,-0.021609,350.202,...,0.616739,59.375000,0.212500,0.787500,18.234795,-72.491564,-70.807951,-0.958372,4.223950,4
3,1004,2000-05-31,5080.0,AIR,2000.0,2.0,3.120983,1.069792,-0.042786,347.451,...,0.645246,154.166667,0.944444,0.055556,18.944015,-75.280235,-73.157425,-0.819497,2.287927,5
4,1004,2000-06-30,5080.0,AIR,2000.0,2.0,3.120983,1.069792,-0.042786,347.451,...,0.645246,154.166667,0.944444,0.055556,18.944015,-75.280235,-73.157425,-0.819497,2.287927,6


In [12]:
result.shape

(2366356, 31)

In [6]:
inf_breakdown = pd.DataFrame()
inf_breakdown['pct_inf'] = (100 * result.isin([float('inf'), float('-inf')]).sum()) / len(compustat_data)
inf_breakdown

,pct_inf
gvkey,0.0
datadate,0.0
sic,0.0
tic,0.0
year,0.0
quarter,0.0
current_ratio,0.0
quick_ratio,0.0
cash_ratio,0.0
net_working_capital,0.0


In [7]:
company_map = pd.read_csv('./compustat_company_map.xlsx - in.csv')
company_map.head()

,company_number,gvkey
0,50797,177439
1,92318,160317
2,92300,106156
3,45321,141466
4,43665,66636


In [8]:
# mapping from company number to gvkey
mapping_dict = company_map.set_index('company_number')['gvkey'].to_dict()

In [9]:
cri_data['gvkey'] = cri_data['CompNo'].map(mapping_dict)

In [10]:
cri_data = cri_data.dropna()
cri_data.shape # lost about 300k rows from missing mapping values, ie 13%

(1943733, 25)

In [11]:
result.head()

,gvkey,datadate,sic,tic,year,quarter,current_ratio,quick_ratio,cash_ratio,net_working_capital,...,working_capital_turnover,price_to_earnings,dividend_payout_ratio,retention_ratio,gross_margin_ratio,operating_profit_margin,ebitda_margin,debt_service_coverage_ratio,interest_coverage_ratio,mm
0,1004,2000-02-29,5080.0,AIR,2000.0,1.0,2.981531,1.176928,-0.021609,350.202,...,0.616739,59.375000,0.212500,0.787500,18.234795,-72.491564,-70.807951,-0.958372,4.223950,2
1,1004,2000-03-31,5080.0,AIR,2000.0,1.0,2.981531,1.176928,-0.021609,350.202,...,0.616739,59.375000,0.212500,0.787500,18.234795,-72.491564,-70.807951,-0.958372,4.223950,3
2,1004,2000-04-30,5080.0,AIR,2000.0,1.0,2.981531,1.176928,-0.021609,350.202,...,0.616739,59.375000,0.212500,0.787500,18.234795,-72.491564,-70.807951,-0.958372,4.223950,4
3,1004,2000-05-31,5080.0,AIR,2000.0,2.0,3.120983,1.069792,-0.042786,347.451,...,0.645246,154.166667,0.944444,0.055556,18.944015,-75.280235,-73.157425,-0.819497,2.287927,5
4,1004,2000-06-30,5080.0,AIR,2000.0,2.0,3.120983,1.069792,-0.042786,347.451,...,0.645246,154.166667,0.944444,0.055556,18.944015,-75.280235,-73.157425,-0.819497,2.287927,6


In [12]:
# clean compustat
result.rename(columns = {'mm': 'month'}, inplace=True)
result.drop(columns = ['datadate', 'quarter', 'sic'], inplace=True)
result.head()

,gvkey,tic,year,current_ratio,quick_ratio,cash_ratio,net_working_capital,debt_ratio,debt_to_equity_ratio,equity_ratio,...,working_capital_turnover,price_to_earnings,dividend_payout_ratio,retention_ratio,gross_margin_ratio,operating_profit_margin,ebitda_margin,debt_service_coverage_ratio,interest_coverage_ratio,month
0,1004,AIR,2000.0,2.981531,1.176928,-0.021609,350.202,0.273288,0.601468,0.454368,...,0.616739,59.375000,0.212500,0.787500,18.234795,-72.491564,-70.807951,-0.958372,4.223950,2
1,1004,AIR,2000.0,2.981531,1.176928,-0.021609,350.202,0.273288,0.601468,0.454368,...,0.616739,59.375000,0.212500,0.787500,18.234795,-72.491564,-70.807951,-0.958372,4.223950,3
2,1004,AIR,2000.0,2.981531,1.176928,-0.021609,350.202,0.273288,0.601468,0.454368,...,0.616739,59.375000,0.212500,0.787500,18.234795,-72.491564,-70.807951,-0.958372,4.223950,4
3,1004,AIR,2000.0,3.120983,1.069792,-0.042786,347.451,0.279030,0.608989,0.458186,...,0.645246,154.166667,0.944444,0.055556,18.944015,-75.280235,-73.157425,-0.819497,2.287927,5
4,1004,AIR,2000.0,3.120983,1.069792,-0.042786,347.451,0.279030,0.608989,0.458186,...,0.645246,154.166667,0.944444,0.055556,18.944015,-75.280235,-73.157425,-0.819497,2.287927,6


In [13]:
# clean compustat
cri_data.rename(columns = {'mm': 'month'}, inplace=True)
cri_data.rename(columns = {'yyyy': 'year'}, inplace=True)

In [14]:
cri_data.head()

,CompNo,year,month,StkIndx,STInt,dtdlevel,dtdtrend,liqnonfinlevel,liqnonfintrend,ni2talevel,...,liqfinlevel,lqfintrend,DTDmedianFin,DTDmedianNonFin,dummy297fin,StartDate,EventDate,Default,Duration,gvkey
4,15984,1999.0,7,0.162536,0.007322,4.190356,0.000000,0.447325,0.000000e+00,0.00932,...,0.0,0.0,0.0,2.455596,0,1999-03-15,1999-07-31,0,138 days,242606.0
5,15984,1999.0,8,0.379335,0.008205,4.728923,0.538567,0.447325,5.551115e-17,0.00932,...,0.0,0.0,0.0,2.459940,0,1999-03-15,1999-08-31,0,169 days,242606.0
6,15984,1999.0,9,0.261256,0.007744,4.779150,0.100454,0.447325,0.000000e+00,0.00932,...,0.0,0.0,0.0,2.499298,0,1999-03-15,1999-09-30,0,199 days,242606.0
7,15984,1999.0,10,0.255081,0.008686,4.872708,0.280674,0.447325,0.000000e+00,0.00932,...,0.0,0.0,0.0,2.506355,0,1999-03-15,1999-10-31,0,230 days,242606.0
8,15984,1999.0,11,0.193601,0.009534,4.932789,0.240323,0.447325,-5.551115e-17,0.00932,...,0.0,0.0,0.0,2.569107,0,1999-03-15,1999-11-30,0,260 days,242606.0


In [15]:
# left join on gvkey, year, month where left is CRI data, right is compustat
cri_compustat_merged = cri_data.merge(result, how='inner')

In [16]:
cri_compustat_merged.head()

,CompNo,year,month,StkIndx,STInt,dtdlevel,dtdtrend,liqnonfinlevel,liqnonfintrend,ni2talevel,...,day_sales_outstanding,working_capital_turnover,price_to_earnings,dividend_payout_ratio,retention_ratio,gross_margin_ratio,operating_profit_margin,ebitda_margin,debt_service_coverage_ratio,interest_coverage_ratio
0,22043,2015.0,12,-0.007266,-0.010905,2.050583,0.000000,1.555253,0.000000e+00,-0.074437,...,209.926660,0.000000,-3.750000,-0.0,1.0,-5.251700e+06,-2.189250e+07,-2.185540e+07,-1.607898,-25.591837
1,22043,2016.0,1,-0.040079,-0.010317,1.907466,-0.143118,1.555253,-2.220446e-16,-0.074437,...,215.036863,-0.645608,-2.333333,-0.0,1.0,-3.784182e+02,-8.568365e+02,-8.246649e+02,-149041.103448,-297211.000000
2,22043,2016.0,2,-0.081412,-0.010297,1.735881,-0.343170,1.555253,0.000000e+00,-0.074437,...,215.036863,-0.645608,-2.333333,-0.0,1.0,-3.784182e+02,-8.568365e+02,-8.246649e+02,-149041.103448,-297211.000000
3,22043,2016.0,3,-0.003930,-0.010763,1.574717,-0.483492,1.460546,-3.788296e-01,-0.077557,...,216.622596,0.000000,-3.230769,-0.0,1.0,-5.251700e+06,-2.189250e+07,-2.185540e+07,-1.315689,-19.537118
4,22043,2016.0,4,-0.019668,-0.010722,1.463769,-0.443790,1.397408,-3.156913e-01,-0.079636,...,216.622596,0.000000,-3.230769,-0.0,1.0,-5.251700e+06,-2.189250e+07,-2.185540e+07,-1.315689,-19.537118


In [17]:
cri_compustat_merged.shape #lost about 760249 rows, or 39%

(1179414, 50)

In [18]:
len(cri_compustat_merged['gvkey'].unique())

10895

In [19]:
inf_breakdown = pd.DataFrame()
inf_breakdown['pct_inf'] = (100 * cri_compustat_merged.isin([float('inf'), float('-inf')]).sum()) / len(cri_compustat_merged)
inf_breakdown

,pct_inf
CompNo,0.0
year,0.0
month,0.0
StkIndx,0.0
STInt,0.0
dtdlevel,0.0
dtdtrend,0.0
liqnonfinlevel,0.0
liqnonfintrend,0.0
ni2talevel,0.0


In [27]:
nan_breakdown = pd.DataFrame()
nan_breakdown['pct_nan'] = (cri_compustat_merged.isna().sum() / len(cri_compustat_merged)) * 100
nan_breakdown

,pct_nan
CompNo,0.0
year,0.0
month,0.0
StkIndx,0.0
STInt,0.0
dtdlevel,0.0
dtdtrend,0.0
liqnonfinlevel,0.0
liqnonfintrend,0.0
ni2talevel,0.0


In [1]:
cri_compustat_merged.to_pickle('./cri_compustat_merged_v8_6months.pkl')

NameError: name 'cri_compustat_merged' is not defined

In [29]:
cri_compustat_merged

,CompNo,year,month,StkIndx,STInt,dtdlevel,dtdtrend,liqnonfinlevel,liqnonfintrend,ni2talevel,...,day_sales_outstanding,working_capital_turnover,price_to_earnings,dividend_payout_ratio,retention_ratio,gross_margin_ratio,operating_profit_margin,ebitda_margin,debt_service_coverage_ratio,interest_coverage_ratio
0,22043,2015.0,12,-0.007266,-0.010905,2.050583,0.000000,1.555253,0.000000e+00,-0.074437,...,209.926660,0.000000,-3.750000,-0.000000,1.000000,-5.251700e+06,-2.189250e+07,-2.185540e+07,-1.607898,-25.591837
1,22043,2016.0,1,-0.040079,-0.010317,1.907466,-0.143118,1.555253,-2.220446e-16,-0.074437,...,215.036863,-0.645608,-2.333333,-0.000000,1.000000,-3.784182e+02,-8.568365e+02,-8.246649e+02,-149041.103448,-297211.000000
2,22043,2016.0,2,-0.081412,-0.010297,1.735881,-0.343170,1.555253,0.000000e+00,-0.074437,...,215.036863,-0.645608,-2.333333,-0.000000,1.000000,-3.784182e+02,-8.568365e+02,-8.246649e+02,-149041.103448,-297211.000000
3,22043,2016.0,3,-0.003930,-0.010763,1.574717,-0.483492,1.460546,-3.788296e-01,-0.077557,...,216.622596,0.000000,-3.230769,-0.000000,1.000000,-5.251700e+06,-2.189250e+07,-2.185540e+07,-1.315689,-19.537118
4,22043,2016.0,4,-0.019668,-0.010722,1.463769,-0.443790,1.397408,-3.156913e-01,-0.079636,...,216.622596,0.000000,-3.230769,-0.000000,1.000000,-5.251700e+06,-2.189250e+07,-2.185540e+07,-1.315689,-19.537118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1179409,93052,2013.0,9,-0.009545,-0.007274,4.892001,1.212125,-0.402310,-8.963039e-03,-0.001001,...,0.000000,-2.837200,64.300000,1.500000,-0.500000,6.804920e+01,3.344428e+01,6.252388e+01,0.100118,15.255908
1179410,93052,2013.0,10,0.030578,-0.007554,4.870839,0.330139,-0.423252,1.197909e-02,-0.001054,...,0.000000,-2.837200,64.300000,1.500000,-0.500000,6.804920e+01,3.344428e+01,6.252388e+01,0.100118,15.255908
1179411,93052,2013.0,11,0.026269,-0.007414,4.842565,0.113506,-0.457056,-1.823057e-01,-0.000853,...,0.000000,-2.837200,64.300000,1.500000,-0.500000,6.804920e+01,3.344428e+01,6.252388e+01,0.100118,15.255908
1179412,93052,2013.0,12,0.023256,-0.007519,4.835230,-0.059513,-0.490860,-1.485018e-01,-0.000653,...,0.000000,-1.877653,96.200000,3.000000,-2.000000,6.061403e+01,1.853357e+01,5.433714e+01,0.046131,12.713643


In [ ]:
yfinance = pd.read_pickle("yfinance_sortino_MACD_RSI.pkl")
yfinance

In [ ]:
yfinance = yfinance.rename(columns={"yyyy":"year"})
yfinance = yfinance.rename(columns={"mm":"month"})
yfinance

In [ ]:
cri_compustat_yfinance = cri_compustat_merged.merge(yfinance, on=['tic','year','month'], how='inner')

In [ ]:
cri_compustat_yfinance.shape[0]

In [ ]:
cri_compustat_yfinance.isna().sum()

In [ ]:
cri_compustat_yfinance.dropna(inplace=True)

In [ ]:
cri_compustat_yfinance.to_pickle("cri_compustat_v8_yfinance.pkl")